In [85]:
import pandas as pd
import numpy as np

<h2 style='color:#00A'>O código adiante obtém os arquivos csv, por tipo (rat, rat_evetivo, rat_viatura e rat_produtividade) e os ordena por data</h2>

In [68]:
files_names

{'rat': ['REDS_RAT_20200411_20200420.csv'],
 'ratv': ['REDS_RAT_VIATURAS_20200411_20200420.csv'],
 'rate': ['REDS_RAT_EFETIVOS_20200411_20200420.csv'],
 'ratp': ['REDS_RAT_Produtividade_20200411_20200420.csv']}

In [101]:
# files_names = funcs.get_files_names(only_not_imported=True)


,index,0
0,0,REDS_RAT_20200411_20200420.csv


In [92]:
print('df_rat_db', df_rat_db.shape)
print('df_rat_new', df_rat_new.shape)
print('df_rat_total', df_rat_total.shape)

df_rat_db (104547, 42)
df_rat_new (762, 42)
df_rat_total (105309, 42)


In [94]:
%%time
import pandas as pd
import funcs

files_names = funcs.get_files_names(only_not_imported=True)
print('files ok')

df_rat = funcs.read_files(files_names['rat'])
print('files lidos ok')

df_rat = funcs.data_rat_processing(df_rat)
print('processing df ok')

df_rat['SETOR'] = df_rat.apply(lambda row: funcs.classifica_setor(row), axis=1)
print('classificação setores ok')

funcs.classifica_cia(df_rat)
print('classificação cias ok')

df_rat_db = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
print('df_rat_db criado')

df_rat_total = df_rat_db.append(df_rat)
df_rat_total = df_rat_total.drop_duplicates(subset='RAT.NUM_ATIVIDADE', keep='last')

df_rat_total.to_sql('tbl_rat', 'sqlite:///gdo.db', if_exists='replace', index=False)
print('dados exportados para o db ok')

pd.DataFrame(files_names['rat']).to_sql('tbl_imported_files', 'sqlite:///gdo.db', if_exists='replace')
print('nomes dos arquivos importados, já inseridos no db')

files ok


b'Skipping line 56784: expected 32 fields, saw 33\n'


files lidos ok
processing df ok
classificação setores ok
classificação cias ok
df_rat_db criado
dados exportados para o db ok


/home/wendel/.asdf/installs/python/anaconda3-2019.07/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  dtype=dtype, method=method)


nomes dos arquivos importados, já inseridos no db
CPU times: user 17.7 s, sys: 178 ms, total: 17.9 s
Wall time: 18 s


<h2 style="color:#0A0;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT</h2>
<h3 style="color:#00A; margin:10px">Importa somente os dados do 23º BPM</h3>

<ol style="color:#222">
    <li>Lê o primeiro arquivo da lista e o passa para um DataFrame (df1)</li>
    <li>Lê o segundo arquivo da lista e o passa para outro DataFrame (df2)</li>
    <li>Concatena os dois DataFrames em um terceiro (df_acum)</li>
    <li>Itera sobre os demais arquivos, lendo cada um deles, aplicando strip(), a função de filtro do 23º BPM e depois o adiciona ao DataFrame df_acum</li>
    <li>Retira as duplicatas, pelo campo 'RAT.NUM_ATIVIDADE'</li>
    <li>Adiciona colunas data/hora início e término, tempo em datetime e tempo em inteiro </li>
    <li>Aplica .upper() nas colunas usadas para classificação.</li>
    <li>Classifica por Setores</li>
    <li>Classifica por Cias</li>
    <li>Insere os dados de df_acum no banco de dados gdo.db: (tabela tbl_rat)</li>

</ol>

In [34]:
import sqlite3
    
with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    cursor.execute('delete from tbl_imported_files')
    conn.commit()

In [32]:
df_rat_existing_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 92 to 60637
Data columns (total 42 columns):
RAT.NUM_ATIVIDADE       1571 non-null object
NAT.CODIGO              1571 non-null object
NAT.DESCRICAO           1571 non-null object
DTA_HRA_INCLUSAO        1571 non-null object
DTA_INICIO              1571 non-null object
HRA_INICIO              1571 non-null object
DTA_TERMINO             1571 non-null object
HRA_TERMINO             1571 non-null object
DES_ALVO_EVENTO         1571 non-null object
DES_LUGAR               1571 non-null object
NOM_OPERACAO            1571 non-null object
COD_UNIDADE_SERVICO     1571 non-null object
NOM_UNID_RESPONSAVEL    1571 non-null object
TIPO_LOGRADOURO         1571 non-null object
LOGRADOURO              1571 non-null object
DES_ENDERECO            1571 non-null object
NUM_ENDERECO            1571 non-null object
COMPLEMENTO_ALFA        1571 non-null object
COMPLEMENTO_ENDERECO    1571 non-null object
NUM_COMPLEMENTAR        1571 non-nul

In [27]:
df_rat_db = df_rat_db.append(df_rat_news_rows.iloc[-1])

In [24]:
# a = pd.DataFrame({
#     1:[1,2,3],
#     2:[4,5,6]
# })
# b = pd.DataFrame({
#     1:[9,8,7],
#     2:[6,5,4]
# })
# a.append(b)
a

,1,2
0,1,4
1,2,5
2,3,6


In [28]:
df_rat_db.tail(3)

,RAT.NUM_ATIVIDADE,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,...,DTA_HRA_INICIO_DT,DTA_HRA_TERMINO_DT,TEMPO_DT,TEMPO_INT,MES,ANO,DEZENA,SETOR,CIA,DIA
46791,2020-018889990-001,Y04011,OPERACAO DE FISCALIZACAO DE TRANSITO,20/04/2020,20/04/2020,19:20,20/04/2020,21:25,AUTOMOVEL,VIA DE ACESSO PUBLICA,...,2020-04-20 19:20:00,2020-04-20 21:25:00,7.5e+12,125,4,2020,2,SAO JOSE,139 CIA,NaN
46792,2020-018899415-001,Y09999,OUTROS POLICIAMENTO ESPECIAL,20/04/2020,20/04/2020,20:18,20/04/2020,20:30,MORADOR / VISITANTE DE RESIDENCIA,CASA,...,2020-04-20 20:18:00,2020-04-20 20:30:00,7.2e+11,12,4,2020,2,PORTO VELHO,142 CIA,NaN
60637,2020-018899415-001,Y09999,OUTROS POLICIAMENTO ESPECIAL,20/04/2020,20/04/2020,20:18,20/04/2020,20:30,MORADOR / VISITANTE DE RESIDENCIA,CASA,...,2020-04-20 20:18:00,2020-04-20 20:30:00,0 days 00:12:00,12,4,2020,2,PORTO VELHO,142 CIA,20.0


In [21]:
df_rat_news_rows.tail(3)

,RAT.NUM_ATIVIDADE,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,...,DTA_HRA_INICIO_DT,DTA_HRA_TERMINO_DT,TEMPO_DT,TEMPO_INT,DIA,MES,ANO,DEZENA,SETOR,CIA
60321,2020-018887191-001,Y04011,OPERACAO DE FISCALIZACAO DE TRANSITO,20/04/2020,20/04/2020,18:00,20/04/2020,19:00,AUTOMOVEL,VIA DE ACESSO PUBLICA,...,2020-04-20 18:00:00,2020-04-20 19:00:00,01:00:00,60,20,4,2020,2,CENTRO/PADRE EUSTAQUIO,51 CIA
60378,2020-018889990-001,Y04011,OPERACAO DE FISCALIZACAO DE TRANSITO,20/04/2020,20/04/2020,19:20,20/04/2020,21:25,AUTOMOVEL,VIA DE ACESSO PUBLICA,...,2020-04-20 19:20:00,2020-04-20 21:25:00,02:05:00,125,20,4,2020,2,SAO JOSE,139 CIA
60637,2020-018899415-001,Y09999,OUTROS POLICIAMENTO ESPECIAL,20/04/2020,20/04/2020,20:18,20/04/2020,20:30,MORADOR / VISITANTE DE RESIDENCIA,CASA,...,2020-04-20 20:18:00,2020-04-20 20:30:00,00:12:00,12,20,4,2020,2,PORTO VELHO,142 CIA


In [10]:
df_rat.groupby(['ANO','MES','DEZENA']).count()

RAT.NUM_ATIVIDADE  NAT.CODIGO  NAT.DESCRICAO  \
ANO  MES DEZENA                                                 
0    0   3                       2           2              2   
2014 3   1                       1           1              1   
     5   3                       2           2              2   
     6   2                       1           1              1   
2016 4   2                       1           1              1   
2017 9   3                       1           1              1   
     10  3                       2           2              2   
     11  2                       1           1              1   
         3                       1           1              1   
     12  3                       1           1              1   
2018 1   1                       1           1              1   
         3                       4           4              4   
     2   1                       2           2              2   
     3   1                       2           2              2   
     4   1                       1           1              1   
         2                       1           1              1   
         3                       3           3              3   
     5   2                       2           2              2   
     6   2                       1           1              1   
         3                       1           1              1   
     7   1                       3           3              3   
         2                       1           1              1   
     8   1                       1           1              1   
         3                       2           2              2   
     9   1                       1           1              1   
         2                       2           2              2   
         3                       1           1              1   
     10  1                       3           3              3   
         2                       3           3              3   
         3                       5           5              5   
...                            ...         ...            ...   
2019 6   3                     937         937            937   
     7   1                     898         898            898   
         2                     868         868            868   
         3                    1342        1342           1342   
     8   1                     959         959            959   
         2                     958         958            958   
         3                    1281        1281           1281   
     9   1                     932         932            932   
         2                     921         921            921   
         3                    1120        1120           1120   
     10  1                     917         917            917   
         2                     848         848            848   
         3                    1277        1277           1277   
     11  1                     912         912            912   
         2                     898         898            898   
         3                    1160        1160           1160   
     12  1                     979         979            979   
         2                     952         952            952   
         3                     906         906            906   
2020 1   1                     875         875            875   
         2                     873         873            873   
         3                    1089        1089           1089   
     2   1                     905         905            905   
         2                     922         922            922   
         3                     735         735            735   
     3   1                     854         854            854   
         2                     870         870            870   
         3                     800         800            800   
     4   1                     773         773      

In [83]:
df_rat[
    df_rat['SETOR'] == 'other'
][[
    'MUNICIPIO',
    'LOGRADOURO',
    'DES_ENDERECO',
    'COMPLEMENTO_ENDERECO',
    'NOME_BAIRRO',
    'LOGRADOURO2',
    'DES_ENDERECO2'
]]

,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2
453,DIVINOPOLIS,,VICENTE LUIZ QUADROS,,,,
454,DIVINOPOLIS,,CONQUISTA,,,,
680,DIVINOPOLIS,,JOAO ANTONIO GONCALVES,,,,
8771,ITAUNA,LENHITA,LENHITA,,,,
9905,ITAUNA,MAGNESITA,MAGNESITA,CASA,,,
29633,ITAUNA,,VOVO BENVINDA PARREIRAS,,,,
41207,ITAUNA,,VOVO BENVINDA PARREIRAS,,,,
41714,ITAUNA,,JOAO CONQUISTA,,,,
3777,DIVINOPOLIS,,RIO DA PALMA,TIETE,,,
7281,DIVINOPOLIS,,BRASILINO SERGIO,CASA,,,


In [28]:
# df_rat['MUNICIPIO'].value_counts()
for i in range( len(files_names['rat']) ):
    print(files_names['rat'][i])

REDS_RAT_20200401_20200410.csv
REDS_RAT_20200411_20200420.csv


In [72]:
df_rat['MUNICIPIO'].iloc[59324:59330]

544    BELO HORIZONTE
545    BELO HORIZONTE
546    BELO HORIZONTE
547    BELO HORIZONTE
548    BELO HORIZONTE
549             BETIM
Name: MUNICIPIO, dtype: object

In [69]:
df_rat[
    df_rat['MUNICIPIO'].isna()
]['MUNICIPIO']

59324    NaN
Name: MUNICIPIO, dtype: object

<h2 style="color:#0A0;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT_VEICULOS, RAT_VIATURA e RAT_PRODUTIVIDADE</h2>
<h3 style="color:#00A; margin:10px">Importa somente os dados do 23º BPM</h3>

<ol style="color:#222">
    <li>Importa do Banco de Dados, a tabela rat, para o df_rat, atribuindo RAT.NUM_ATIVIDADE para o index</li>
    <li>Lê o primeiro arquivo da lista e o passa para um DataFrame (dfv1, dfe1, dfp1)</li>
    <li>Lê o segundo arquivo da lista e o passa para outro DataFrame (dfv2, dfe2, dfp2)</li>
    <li>Concatena os dois DataFrames em um terceiro (df_acum)</li>
    <li>Itera sobre os demais arquivos, lendo cada um deles, aplicando strip(), a função de filtro do 23º BPM e depois o adiciona ao DataFrame df_acum</li>    
    <li>Insere os dados no banco de dados</li>        
</ol>    

In [27]:
df_ratv = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
df_ratv.head()

,RAT.NUM_ATIVIDADE,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,...,NOM_UNIDADE_AREA,DIGITADOR,DTA_HRA_INICIO_DT,DTA_HRA_TERMINO_DT,TEMPO_DT,TEMPO_INT,MES,ANO,SETOR,CIA
0,2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,...,53 CIA PM/23 BPM/7 RPM,PM1569110,2018-10-17 18:43:00,2018-10-17 19:00:00,1.020000e+12,17,10,2018,PLANALTO,139 CIA
1,2018-046878708-001,Y12001,FISCALIZACAO DE LOCAIS DE DESMATE,20/10/2018,20/10/2018,12:39,20/10/2018,15:00,FAZENDA,FAZENDA,...,3 PEL/139 CIA PM/23 BPM/7 RPM,PM1502681,2018-10-20 12:39:00,2018-10-20 15:00:00,8.460000e+12,141,10,2018,CLAUDIO,139 CIA
2,2018-047006854-001,Y11006,FISCALIZACAO DE MAUS TRATOS A ANIMAIS,21/10/2018,21/10/2018,13:28,21/10/2018,16:00,FAZENDA,FAZENDA,...,142 CIA PM/23 BPM/7 RPM,PM1502681,2018-10-21 13:28:00,2018-10-21 16:00:00,9.120000e+12,152,10,2018,PORTO VELHO,142 CIA
3,2018-048663517-001,Y12001,FISCALIZACAO DE LOCAIS DE DESMATE,31/10/2018,31/10/2018,07:23,31/10/2018,10:00,FAZENDA,FAZENDA,...,53 CIA PM/23 BPM/7 RPM,PM1502681,2018-10-31 07:23:00,2018-10-31 10:00:00,9.420000e+12,157,10,2018,PLANALTO,139 CIA
4,2018-049254756-001,Y08999,OUTRAS ESCOLTAS,04/11/2018,04/11/2018,08:22,04/11/2018,10:00,CARGAS,VIA DE ACESSO PUBLICA,...,53 CIA PM/23 BPM/7 RPM,PM1326123,2018-11-04 08:22:00,2018-11-04 10:00:00,5.880000e+12,98,11,2018,BOM PASTOR,53 CIA


In [7]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfv1 = pd.read_csv('files/RAT/' + rats_viatura_files[0], error_bad_lines=False, sep='|')
# dfv1 = dfv1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv1 = dfv1[dfv1['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfv2 = pd.read_csv('files/RAT/' + rats_viatura_files[1], error_bad_lines=False, sep='|')
# dfv2 = dfv2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv2 = dfv2[dfv2['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfv_acum = pd.concat([dfv1, dfv2])


# for rat_viatura_file in rats_viatura_files[2:]:
#     dfv_aux = pd.read_csv('files/RAT/'+rat_viatura_file, sep='|')
#     dfv_aux = dfv_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfv_aux = dfv_aux[dfv_aux['NUM_ATIVIDADE'].isin(df_rat.index)]
#     dfv_acum = pd.concat([dfv_acum, dfv_aux])
    
# dfv_acum.to_sql('tbl_rat_veiculo', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del dfv_acum
# del df_rat

In [9]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfe1 = pd.read_csv('files/RAT/' + rats_efetivo_files[0], error_bad_lines=False, sep='|')
# dfe1 = dfe1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfe1 = dfe1[dfe1['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfe2 = pd.read_csv('files/RAT/' + rats_efetivo_files[1], error_bad_lines=False, sep='|')
# dfe2 = dfe2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfe2 = dfe2[dfe2['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfe_acum = pd.concat([dfe1, dfe2])


# for rat_efetivo_file in rats_efetivo_files[2:]:
#     dfe_aux = pd.read_csv('files/RAT/'+rat_efetivo_file, sep='|')
#     dfe_aux = dfe_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfe_aux = dfe_aux[dfe_aux['NUM_ATIVIDADE'].isin(df_rat.index)]
#     dfe_acum = pd.concat([dfe_acum, dfe_aux])
    
# dfe_acum.to_sql('tbl_rat_efetivo', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del dfe_acum
# del df_rat

In [10]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfp1 = pd.read_csv('files/RAT/' + rats_produtividade_files[0], error_bad_lines=False, sep='|')
# dfp1 = dfp1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfp1 = dfp1[dfp1['RAT.NUM_ATIVIDADE'].isin(df_rat.index)]

# dfp2 = pd.read_csv('files/RAT/' + rats_produtividade_files[1], error_bad_lines=False, sep='|')
# dfp2 = dfp2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfp2 = dfp2[dfp2['RAT.NUM_ATIVIDADE'].isin(df_rat.index)]

# dfp_acum = pd.concat([dfp1, dfp2])


# for rat_produtividade_file in rats_produtividade_files[2:]:
#     dfp_aux = pd.read_csv('files/RAT/'+rat_produtividade_file, sep='|')
#     dfp_aux = dfp_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfp_aux = dfp_aux[dfp_aux['RAT.NUM_ATIVIDADE'].isin(df_rat.index)]
#     dfp_acum = pd.concat([dfp_acum, dfp_aux])
    
# dfp_acum.to_sql('tbl_rat_produtividade', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del df_rat
# del dfp_acum